In [1]:
# Parameters
image_path = "input_image.png"


In [2]:
try:
    image_path
except NameError:
    image_path = 'sample-flowcharts/fc4.jpeg'

In [3]:
import cv2
import numpy as np
import math
import os
from matplotlib import pyplot as plt

In [4]:
# Helper methods
def denoiseAndFill(m, thres):
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    denoise = np.zeros_like(m)
    for contour in contours:
        if cv2.contourArea(contour) > thres:
            cv2.drawContours(denoise, [contour], -1, 255, thickness=cv2.FILLED)
    return denoise

def rotate(m, lines):
    angles = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            angle = math.atan2(y2 - y1, x2 - x1)
            angles.append(angle)
    angles.sort()
    majority_angles = findMajority(np.array(angles), 10)
    angle = np.median(majority_angles)
    degree = math.degrees(angle)

    # Rotate image
    (h, w) = m.shape[:2]
    center = (w // 2, h // 2)
    rotMat = cv2.getRotationMatrix2D(center, degree, 1.0)
    rotated = cv2.warpAffine(m, rotMat, (w, h), flags=cv2.INTER_LINEAR)
    return rotated

def findMajority(input_angles, splits):
    pi = math.pi
    bins = np.linspace(-pi/2, pi/2, splits+1)
    hist, bin_edges = np.histogram(input_angles, bins)
    max_bin = np.argmax(hist)
    mask = (input_angles >= bin_edges[max_bin]) & (input_angles < bin_edges[max_bin+1])
    majority = input_angles[mask]
    return majority

def dilate(m, r):
    se = seGen(r)
    dilated = cv2.dilate(m, se)
    return dilated

def erode(m, r):
    se = seGen(r)
    eroded = cv2.erode(m, se)
    return eroded

def seGen(radius):
    diameter = 2 * radius + 1
    se = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (diameter, diameter))
    return se

def genRectAndDiam(m):
    contours, _ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    rectangle = np.zeros_like(m)
    diamond = np.zeros_like(m)
    for contour in contours:
        area = cv2.contourArea(contour)
        x, y, w, h = cv2.boundingRect(contour)
        rect_area = w * h
        if area / rect_area > 0.75:
            cv2.drawContours(rectangle, [contour], -1, 255, thickness=cv2.FILLED)
        else:
            cv2.drawContours(diamond, [contour], -1, 255, thickness=cv2.FILLED)
    return rectangle, diamond

def plot_image(img, title):
    plt.figure()
    plt.imshow(img, cmap="gray")
    plt.axis('off')
    plt.title(title)
    plt.show()

In [5]:
def prepro(mat):
    plot_image(mat, "Original Image")
    rows, cols = mat.shape[:2]

    # Convert to grayscale
    gray = cv2.cvtColor(mat, cv2.COLOR_RGB2GRAY)
    plot_image(gray, "Grayscale Image")

    # Adaptive thresholding (binarization)
    bina = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 65, 40)
    plot_image(bina, "Binarized Image")

    # Bitwise inversion
    bina = cv2.bitwise_not(bina)
    plot_image(bina, "Bitwise Inverted Image")


    # Denoise and fill
    denoise = denoiseAndFill(bina, 100)
    plot_image(denoise, "After denoising and fill")

    # Edge detection
    edges = cv2.Canny(denoise, 100, 100, apertureSize=3, L2gradient=False)
    plot_image(edges, "Detected Edges")

    # Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180.0, 50, minLineLength=30, maxLineGap=5)
    if lines is not None and len(lines) > 0:
        rotated = rotate(denoise, lines)
        substitute = rotated.copy()
        rows, cols = substitute.shape[:2]
    else:
        substitute = denoise.copy()
    plot_image(substitute, "Rotated Image")

    # Fill small regions
    fill = denoiseAndFill(substitute, 100)

    # Morphological opening to eliminate arrows
    seOpen = seGen(10)
    opened = cv2.morphologyEx(fill, cv2.MORPH_OPEN, seOpen)
    opened = denoiseAndFill(opened, 100)
    plot_image(opened, "Opened Image")

    # Compute difference to get arrows
    diff = cv2.absdiff(fill, opened)
    plot_image(diff, "Arrows")

    # Remove small areas generated by opening
    remv = denoiseAndFill(diff, 50)
    arro = dilate(remv, 10)
    plot_image(arro, "Enhanced Arrows")

    # Extract rectangles and diamonds
    blob = cv2.absdiff(fill, remv)

    # Find circles
    eroded_blob = erode(blob, 10)
    diff_blob = cv2.absdiff(blob, eroded_blob)
    copy_blob = diff_blob.copy()
    contours, _ = cv2.findContours(diff_blob, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        temp = np.zeros_like(diff_blob)
        cv2.drawContours(temp, [contour], -1, 255, 10)
        circles = cv2.HoughCircles(temp, cv2.HOUGH_GRADIENT, 2, substitute.shape[0]/4, param1=200, param2=100, minRadius=0, maxRadius=0)
        if circles is not None and len(circles) > 0:
            cv2.fillPoly(copy_blob, [contour], 0)
    circ = cv2.absdiff(diff_blob, copy_blob)

    # Distinguish rectangles and diamonds
    rect, diam = genRectAndDiam(copy_blob)

    # Erode
    eroded_rect = erode(rect, 10)
    eroded_diam = erode(diam, 10)

    # Get final rectangles and diamonds
    rect = cv2.absdiff(rect, eroded_rect)
    diam = cv2.absdiff(diam, eroded_diam)

    # Final result
    result = [rect, diam, arro, circ]
    plot_image(rect, "Rectangles")
    plot_image(diam, "Diamonds")
    plot_image(circ, "Circles")
    plot_image(arro, "Arrows")
    return result

In [6]:
# image_path = "sample-flowcharts/fc4.jpeg"
fc = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
result = prepro(fc)

/var/folders/zf/tlb2078n343gt63yyzg050lm0000gn/T/ipykernel_81106/3377694811.py:71: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [7]:
def detect_and_draw_contours_exact(binary_image, canvas, color):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        cv2.drawContours(canvas, [contour], -1, color, thickness=cv2.FILLED)        

In [8]:
[rectangle, diamond, circle, arrows] = result

assert circle.shape == rectangle.shape == diamond.shape == arrows.shape
height, width = circle.shape
canvas = np.zeros((height, width, 4), dtype=np.uint8)

circle_color = (255, 0, 0, 255)    # Red
rectangle_color = (0, 255, 0, 255)  # Green
diamond_color = (0, 0, 255, 255)   # Blue
arrows_color = (255, 255, 0, 255)  # Yellow

detect_and_draw_contours_exact(circle, canvas, circle_color)
detect_and_draw_contours_exact(rectangle, canvas, rectangle_color)
detect_and_draw_contours_exact(diamond, canvas, diamond_color)
detect_and_draw_contours_exact(arrows, canvas, arrows_color)

cv2.imwrite('output-flowchart.png', canvas)
plt.imshow(cv2.cvtColor(canvas, cv2.COLOR_RGBA2RGB))
plt.title('Combined Shapes with Retained Positions')
plt.axis('off')
plt.show()

/var/folders/zf/tlb2078n343gt63yyzg050lm0000gn/T/ipykernel_81106/4195007297.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
